# A demo sample for citizen science

Melissa Graham <br>
Fri Dec 16 2022 <br>
Weekly 2022_40 <br>

Based on tutorial notebooks 03a and 07a.

Identify a sample of 100 "low-z SNIa-like" `DiaObjects`.

Create two derived data products per `DiaObject`:
1. multiband lightcurves using the `DiaSources` (detections, not forced photometry)
2. 20x20 arcsec i-band cutouts of the deepCoadds (to see host galaxies)

Output files stored in "citsci_sample_files/" with prefix "lc_" for lightcurves and "co_" for cutouts.

## Imports

In [1]:
import gc
import time
import numpy
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM

import lsst.afw.display as afwDisplay
from lsst.daf.butler import Butler
from lsst.rsp import get_tap_service
import lsst.geom as geom
from lsst.rsp import get_tap_service

## Services, parameters settings, functions

In [2]:
afwDisplay.setDefaultBackend('matplotlib')

cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

plt.style.use('tableau-colorblind10')
plot_filter_labels = ['u', 'g', 'r', 'i', 'z', 'y']
plot_filter_colors = {'u': '#56b4e9', 'g': '#008060', 'r': '#ff4000',
                      'i': '#850000', 'z': '#6600cc', 'y': '#000000'}
plot_filter_symbols = {'u': 'o', 'g': '^', 'r': 'v', 'i': 's', 'z': '*', 'y': 'p'}

In [3]:
service = get_tap_service()

In [4]:
butler = Butler('dp02', collections='2.2i/runs/DP0.2')

In [5]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector

In [6]:
def cutout_coadd(butler, ra, dec, band='r', datasetType='deepCoadd',
                 skymap=None, cutoutSideLength=51, **kwargs):
    """
    Produce a cutout from a coadd at the given ra, dec position.

    Adapted from DC2 tutorial notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Servant providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    band: string
        Filter of the image to load
    datasetType: string ['deepCoadd']
        Which type of coadd to load.  Doesn't support 'calexp'
    skymap: lsst.afw.skyMap.SkyMap [optional]
        Pass in to avoid the Butler read.  Useful if you have lots of them.
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)

    if skymap is None:
        skymap = butler.get("skyMap")

    # Look up the tract, patch for the RA, Dec
    tractInfo = skymap.findTract(radec)
    patchInfo = tractInfo.findPatch(radec)
    xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    patch = tractInfo.getSequentialPatchIndex(patchInfo)

    coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
    parameters = {'bbox': bbox}

    cutout_image = butler.get(datasetType, parameters=parameters,
                              dataId=coaddId)

    return cutout_image

## Identify "low-z SNIa-like" DiaObjects

In [7]:
redshift_min = 0.1
redshift_max = 0.3

snia_peak_mag = -19.0
snia_peak_mag_range = 0.5

snia_peak_mr_min = cosmo.distmod(redshift_min).value + snia_peak_mag - snia_peak_mag_range
snia_peak_mr_max = cosmo.distmod(redshift_max).value + snia_peak_mag + snia_peak_mag_range

snia_peak_mg_max = 24.0
snia_peak_mi_max = 24.0

snia_ampl_mr_min = 1.5
snia_ampl_mr_max = 5.5

snia_nDiaSources_min = 15
snia_nDiaSources_max = 100

snia_duration_min = 50
snia_duration_max = 300

In [8]:
%%time
results = service.search("SELECT ra, decl, diaObjectId "
                         "FROM dp02_dc2_catalogs.DiaObject "
                         "WHERE nDiaSources > "+str(snia_nDiaSources_min)+" "
                         "AND nDiaSources < "+str(snia_nDiaSources_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMax) > "+str(snia_peak_mr_min)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMax) < "+str(snia_peak_mr_max)+" "
                         "AND scisql_nanojanskyToAbMag(gPSFluxMax) < "+str(snia_peak_mg_max)+" "
                         "AND scisql_nanojanskyToAbMag(iPSFluxMax) < "+str(snia_peak_mi_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMin)"
                         " - scisql_nanojanskyToAbMag(rPSFluxMax) < "+str(snia_ampl_mr_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMin)"
                         " - scisql_nanojanskyToAbMag(rPSFluxMax) > "+str(snia_ampl_mr_min)+" ",
                         maxrec=3000)
DiaObjs = results.to_table()
del results

CPU times: user 41.9 ms, sys: 0 ns, total: 41.9 ms
Wall time: 19.9 s


In [9]:
my_DiaObjId_string = '('
for i in range(len(DiaObjs)):
    my_DiaObjId_string += str(DiaObjs['diaObjectId'][i])
    if i < len(DiaObjs)-1:
        my_DiaObjId_string += ', '
    else:
        my_DiaObjId_string += ')'

In [10]:
%%time
results = service.search("SELECT ra, decl, diaObjectId, diaSourceId, "
                         "filterName, midPointTai, "
                         "scisql_nanojanskyToAbMag(psFlux) AS psAbMag "
                         "FROM dp02_dc2_catalogs.DiaSource "
                         "WHERE diaObjectId IN "+my_DiaObjId_string)
DiaSrcs = results.to_table()
del results

CPU times: user 1.84 s, sys: 19.9 ms, total: 1.86 s
Wall time: 18.7 s


In [11]:
DiaObjs['duration'] = numpy.zeros(len(DiaObjs), dtype='float')
for i in range(len(DiaObjs)):
    tx = numpy.where(DiaSrcs['diaObjectId'] == DiaObjs['diaObjectId'][i])[0]
    DiaObjs['duration'][i] = numpy.max(DiaSrcs['midPointTai'][tx]) \
                             - numpy.min(DiaSrcs['midPointTai'][tx])

In [12]:
sel_sne = numpy.where((DiaObjs['duration'] > snia_duration_min)
                       & (DiaObjs['duration'] < snia_duration_max))[0]
print(len(sel_sne))

171


## Make PNG lightcurves

In [13]:
%%time
outpath = 'citsci_sample_files/'
temp = range(100)

for sel in sel_sne[temp]:
    fnm = outpath + 'lc_'+ str(DiaObjs['diaObjectId'][sel])
    fig = plt.figure(figsize=(4, 3))
    
    tx = numpy.where(DiaSrcs['diaObjectId'] == DiaObjs['diaObjectId'][sel])[0]
    mx = numpy.argmin(DiaSrcs['psAbMag'][tx])
    t0 = DiaSrcs['midPointTai'][tx[mx]]
    y1 = numpy.max(DiaSrcs['psAbMag'][tx]) + 0.2
    y2 = numpy.min(DiaSrcs['psAbMag'][tx]) - 0.2
    del tx, mx
    
    for f, filt in enumerate(plot_filter_labels):
        fx = numpy.where((DiaSrcs['diaObjectId'] == DiaObjs['diaObjectId'][sel]) \
                         & (DiaSrcs['filterName'] == filt))[0]
        plt.plot(DiaSrcs['midPointTai'][fx] - t0,
                 DiaSrcs['psAbMag'][fx],
                 plot_filter_symbols[filt], ms=10, mew=0, alpha=0.5,
                 color=plot_filter_colors[filt])
        del fx
        
    plt.ylim([y1, y2])
    plt.xlabel('days')
    plt.ylabel('magnitude')
    plt.title(str(DiaObjs['diaObjectId'][sel]))
    plt.tight_layout()
    plt.savefig(fnm)
    plt.close()
    
    del t0, y1, y2

CPU times: user 11.9 s, sys: 13.5 s, total: 25.3 s
Wall time: 12.9 s


## Make PNG cutouts

In [14]:
%%time
outpath = 'citsci_sample_files/'
temp = range(100)

for sel in sel_sne[temp]:
    fnm = outpath + 'co_' + str(DiaObjs['diaObjectId'][sel])
    ra = DiaObjs['ra'][sel]
    dec = DiaObjs['decl'][sel]
    
    cutout_image = cutout_coadd(butler, ra, dec, band='i', 
                                datasetType='deepCoadd',
                                cutoutSideLength=101)

    fig = plt.figure(figsize=(4, 4))
    afw_display = afwDisplay.Display(frame=fig)
    afw_display.scale('asinh', 'zscale')
    afw_display.mtv(cutout_image.image)
    
    cutout_wcs = cutout_image.getWcs()
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    xy = geom.PointI(cutout_wcs.skyToPixel(radec))
    
    afw_display.dot('x', xy.getX(), xy.getY(), size=1, ctype='orange')
    plt.gca().axis('off')
    plt.title(str(DiaObjs['diaObjectId'][sel]))
    plt.savefig(fnm)
    plt.close()
    
    del fnm, ra, dec
    del cutout_image, cutout_wcs, radec, xy
    remove_figure(fig)

CPU times: user 3min 19s, sys: 1min 6s, total: 4min 25s
Wall time: 5min 24s
